In [1]:
from music21 import converter, instrument, note, stream, chord
import glob
import numpy as np
import pickle
from keras.utils import np_utils

Using TensorFlow backend.


## Reading midi file

In [2]:
midi = converter.parse("midi_songs/EyesOnMePiano.mid")

In [3]:
midi

<music21.stream.Score 0x16292638e88>

In [4]:
midi.show('midi')

In [5]:
midi.show('text')

{0.0} <music21.stream.Part 0x1629263c748>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=95.0>
    {0.0} <music21.key.Key of D major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x1629265a148>
        {0.0} <music21.note.Note A>
        {0.0} <music21.note.Note A>
        {0.6667} <music21.note.Note G>
        {1.5} <music21.note.Note D>
        {2.0} <music21.note.Note A>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note D>
        {3.5} <music21.note.Note A>
        {4.0} <music21.note.Note A>
        {6.0} <music21.note.Note A>
        {8.0} <music21.note.Note F#>
        {10.0} <music21.chord.Chord F#4 C#5>
        {11.0} <music21.note.Note A>
        {11.6667} <music21.note.Note D>
        {13.3333} <music21.note.Note D>
        {13.6667} <music21.note.Note C#>
        {14.0} <music21.note.Note D>
        {15.0} <music21.note.Note D>
        {16.0} <music21.note.Note G>
        {16.6667

In [6]:
elements = midi.flat.notes

In [7]:
print(len(elements))

1421


## Preprocessing all files

In [8]:
 notes = []

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    print("parsing %s"%file)
    
    elements = midi.flat.notes
    
    for ele in elements:
        if isinstance(ele,note.Note):
            notes.append(str(ele.pitch))
            
        elif isinstance(ele,chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

parsing midi_songs\0fithos.mid
parsing midi_songs\8.mid
parsing midi_songs\ahead_on_our_way_piano.mid
parsing midi_songs\AT.mid
parsing midi_songs\balamb.mid
parsing midi_songs\bcm.mid
parsing midi_songs\BlueStone_LastDungeon.mid
parsing midi_songs\braska.mid
parsing midi_songs\caitsith.mid
parsing midi_songs\Cids.mid
parsing midi_songs\cosmo.mid
parsing midi_songs\costadsol.mid
parsing midi_songs\dayafter.mid
parsing midi_songs\decisive.mid
parsing midi_songs\dontbeafraid.mid
parsing midi_songs\DOS.mid
parsing midi_songs\electric_de_chocobo.mid
parsing midi_songs\Eternal_Harvest.mid
parsing midi_songs\EyesOnMePiano.mid
parsing midi_songs\ff11_awakening_piano.mid
parsing midi_songs\ff1battp.mid
parsing midi_songs\FF3_Battle_(Piano).mid
parsing midi_songs\FF3_Third_Phase_Final_(Piano).mid
parsing midi_songs\ff4-airship.mid
parsing midi_songs\Ff4-BattleLust.mid
parsing midi_songs\ff4-fight1.mid
parsing midi_songs\ff4-town.mid
parsing midi_songs\FF4.mid
parsing midi_songs\ff4pclov.mid
par

In [9]:
len(notes)

59349

In [10]:
with open("notes", 'wb') as filepath:
    pickle.dump(notes,filepath)

In [11]:
with open("notes", 'rb') as f:
    notes= pickle.load(f)

In [12]:
n_vocab = len(set(notes))

In [13]:
print("Total notes-",len(notes))
print("Unique notes-",n_vocab)

Total notes- 59349
Unique notes- 359


## Preparing sequential data for LSTM

In [14]:
sequence_length = 100

In [15]:
pitch_names = sorted(set(notes))

In [16]:
element_to_int = dict((ele,num) for num,ele in enumerate(pitch_names))
element_to_int

{'0': 0,
 '0+1': 1,
 '0+1+5': 2,
 '0+1+6': 3,
 '0+2': 4,
 '0+2+3+7': 5,
 '0+2+4+7': 6,
 '0+2+5': 7,
 '0+2+6': 8,
 '0+2+7': 9,
 '0+3': 10,
 '0+3+5': 11,
 '0+3+5+8': 12,
 '0+3+6': 13,
 '0+3+6+8': 14,
 '0+3+6+9': 15,
 '0+3+7': 16,
 '0+4': 17,
 '0+4+5': 18,
 '0+4+6': 19,
 '0+4+7': 20,
 '0+5': 21,
 '0+5+6': 22,
 '0+6': 23,
 '1': 24,
 '1+2': 25,
 '1+2+4+6+8+10': 26,
 '1+2+6': 27,
 '1+2+6+8': 28,
 '1+3': 29,
 '1+3+5': 30,
 '1+3+5+8': 31,
 '1+3+6': 32,
 '1+3+7': 33,
 '1+3+8': 34,
 '1+4': 35,
 '1+4+6': 36,
 '1+4+6+9': 37,
 '1+4+7': 38,
 '1+4+7+10': 39,
 '1+4+7+9': 40,
 '1+4+8': 41,
 '1+5': 42,
 '1+5+8': 43,
 '1+5+9': 44,
 '1+6': 45,
 '1+7': 46,
 '10': 47,
 '10+0': 48,
 '10+0+2+5': 49,
 '10+0+3': 50,
 '10+0+4': 51,
 '10+0+5': 52,
 '10+1': 53,
 '10+1+3': 54,
 '10+1+3+5+6': 55,
 '10+1+3+6': 56,
 '10+1+4': 57,
 '10+1+4+6': 58,
 '10+1+5': 59,
 '10+11': 60,
 '10+11+3': 61,
 '10+11+3+5': 62,
 '10+2': 63,
 '10+2+3': 64,
 '10+2+4': 65,
 '10+2+5': 66,
 '10+3': 67,
 '11': 68,
 '11+0': 69,
 '11+0+4': 70,
 

In [17]:
network_input = []
network_output = [] 


In [18]:
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length] 
    seq_out = notes[i+sequence_length]
    
    network_input.append([element_to_int[ch] for ch in seq_in])
    network_output.append(element_to_int[seq_out])

In [19]:

n_patterns = len(network_input)
print(n_patterns)

59249


In [20]:
network_input = np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)

(59249, 100, 1)


In [21]:
normalised_network_input = network_input/float(n_vocab)

In [22]:
network_output = np_utils.to_categorical(network_output)


In [23]:
print(normalised_network_input.shape)
print(network_output.shape)

(59249, 100, 1)
(59249, 359)


## Model

In [24]:
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint

In [25]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

In [26]:
model.compile(loss="categorical_crossentropy", optimizer="adam")



In [27]:
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [47]:
# checkpoint = ModelCheckpoint("model.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')
# model_his = model.fit(normalised_network_input, network_output, epochs=10, batch_size=64, callbacks=[checkpoint])

In [28]:
model = load_model("model.hdf5")

## Predictions

In [29]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] 
    network_input.append([element_to_int[ch] for ch in seq_in])

In [30]:
start = np.random.randint(len(network_input) - 1)

int_to_ele = dict((num, ele) for num, ele in enumerate(pitch_names))

# Taking initial pattern 
pattern = network_input[start]
prediction_output = []

# Generating 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))  
    prediction_input = prediction_input/float(n_vocab) 
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    pattern.append(idx)
    pattern = pattern[1:]

In [31]:
print(prediction_output)

['3+5', '5', 'C4', '5', 'E-4', 'F4', '3+8', '0+5', '3+8', '0+5', '2+7', '2+5', '1+6', '2+5', '0+5', '3+5', '0+5', '3+5', '10+3', '5', '0+5', '5', '0+5', '0+5', '0+5', '10+3', '2+5', '0+5', '2+5', '3+5', '0+5', '3+5', '3+8', '5', '0+5', '5', '0+5', '0+5', '2+5', '2+5', '0+5', 'B5', '0+5', '2+5', 'C6', '2+5', 'G#5', 'F5', 'G#4', '0+2+3+7', '0+5', 'A4', '0+5', 'F5', '2+5', 'D5', '2+5', 'C5', 'A4', '5+7+8+0', '3+5', 'C4', '3+5', '5', '5', '5+7+0', '7+0', '7+0', '9+0', '9+0', '3+5+10', '7+0', '7+0', '5+7+0', '9+0', '9+0', '8+10+3', '10+0', '10+0', '10+0+5', '0', '0', '10+0', '10+0', '5+7+0', '0', '0', 'B-2', 'F#3', 'B-4', 'E4', 'C#5', 'G#5', 'A4', 'F3', 'E-4', 'C5', 'G5', 'E3', 'G#4', 'D4', 'B4', 'F#5', 'G4', 'E-3', 'C#4', 'B-4', 'F5', 'D3', 'F#4', 'C4', 'A4', 'E5', 'F4', 'C#3', 'B3', 'G#4', 'E-5', 'C3', 'E4', 'B-3', 'G4', 'D5', 'F2', '0+5', '9+0+2', '0+5', 'B-2', '8+10+2', '0+5', 'F2', '9+0+2', '0+5', 'B-2', '0+5', 'F2', 'A4', '0+5', 'F2', '0+5', '0+5', '0+5', '2+5', 'F2', '0+5', 'F4', 'C3

## Create midi file

In [32]:
offset = 0 
output_notes = []

for pattern in prediction_output:
    
    # if pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) 
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

In [33]:

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid")

'test_output.mid'

In [34]:
midi_stream.show('midi')